In [15]:
import subprocess
import os
import numpy as np

# Dependencies
import rasterio
from scipy.interpolate import griddata
from scipy.ndimage import zoom
from pyproj import Proj, Transformer

In [2]:
# Setup filepaths
home_dir = os.path.expanduser('~')
username = os.path.basename(home_dir)
gdata_dir = os.path.join("/g/data/xe2", username)
scratch_dir = os.path.join('/scratch/xe2', username)
paddockTS_dir = os.path.join(home_dir, "Projects/PaddockTS")
print("username:", username)
print("home_dir:", home_dir)
print("gdata_dir:", gdata_dir)
print("scratch_dir:", scratch_dir)

username: cb8590
home_dir: /home/147/cb8590
gdata_dir: /g/data/xe2/cb8590
scratch_dir: /scratch/xe2/cb8590


In [3]:
# Local imports
os.chdir(paddockTS_dir)
from DAESIM_preprocess.slga_soils import create_bbox

In [4]:
# Choosing location
lat, lon = -34.3890427, 148.469499
buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total

stub = "MILG_1km"
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = os.path.join(scratch_dir, "tmp")

In [5]:
bbox = create_bbox(lat, lon, buffer)
bbox

[148.46449900000002, -34.3940427, 148.474499, -34.384042699999995]

In [21]:
def transform_bbox(bbox=[148.464499, -34.394042, 148.474499, -34.384042], inputEPSG="EPSG:4326", outputEPSG="EPSG:3857"):
    transformer = Transformer.from_crs(inputEPSG, outputEPSG)
    x1,y1 = transformer.transform(bbox[1], bbox[0])
    x2,y2 = transformer.transform(bbox[3], bbox[2])
    return (x1, y1, x2, y2)

bbox_3857 = transform_bbox()
bbox_3857

(16526992.42955847,
 -4081835.7602158766,
 16528105.624466406,
 -4080486.7952851765)

In [ ]:
def run_gdalwarp(bbox=[148.464499, -34.394042, 148.474499, -34.3840426], xml="mapzen_wms.xml", filename="output.tif"):
    """Use gdalwarp to download a tif from terrain tiles"""
    min_x, min_y, max_x, max_y = bbox
    command = [
        "gdalwarp",
        "-s_srs", "EPSG:4326",
        "-t_srs", "EPSG:3857"
        "-of", "GTiff",
        "-te", str(min_x), str(min_y), str(max_x), str(max_y),
        xml, filename
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    print(f"Downloaded {filename}")

    print("STDOUT:", result.stdout, flush=True)
    print("STDERR:", result.stderr, flush=True)



result = run_gdalwarp(bbox=coords)

# Above here is new code

In [28]:
def transform_coords(c1, c2, inputEPSG, outputEPSG):
    x1,y1 = transform_proj(c1[0], c1[1], inputEPSG, outputEPSG)
    x2,y2 = transform_proj(c2[0], c2[1], inputEPSG, outputEPSG)
    return (x1, y1, x2, y2)

def transform_proj(x, y, inputEPSG, outputEPSG):
    in_str = 'epsg:' + str(inputEPSG)
    out_str = 'epsg:' + str(outputEPSG)
    tf = Transformer.from_crs(in_str, out_str)
    return tf.transform(x, y)


def create_bounding_box(lat, lon, distance_km):
    """Use google maps coordinates to create bounding box for Terrain Tiles"""
    wgs84 = Proj(init='epsg:4326')  # WGS 84
    web_mercator = Proj(init='epsg:3857')  # Web Mercator
    
    # Convert the center point to EPSG:3857
    x_center, y_center = transform(wgs84, web_mercator, lon, lat)
    
    # Calculate the offset in meters (10 km = 10,000 meters)
    offset = distance_km * 1000 / 2  # half distance for each direction
    
    # Define the bounding box in EPSG:3857
    min_x = x_center - offset
    max_x = x_center + offset
    min_y = y_center - offset
    max_y = y_center + offset
    
    return min_x, min_y, max_x, max_y

In [38]:
spring_valley_single = "-35.280716, 149.021031"
coord = parse_coord(spring_valley_single)
coords = create_bounding_box(coord[0], coord[1], distance_km=3)
coords

Parsing coord: -35.280716, 149.021031
Parsing single coord
Parsed coords: ['-35.280716', ' 149.021031']


/g/data/xe2/John/geospatenv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/g/data/xe2/John/geospatenv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/jobfs/123114450.gadi-pbs/ipykernel_3430486/434968372.py:19: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-py

(16587445.288408635,
 -4203595.066255477,
 16590445.288408635,
 -4200595.066255477)

In [39]:
c1 = [-35.297579, 149.003879]
c2 = [-35.280223, 149.018751]

# Terrain tiles API requires EPSG:3857
bbox_3857 = transform_coords(c1, c2, 4326, 3857)
c1, c2 = (bbox_3857[1], bbox_3857[0]), (bbox_3857[3], bbox_3857[2]) 
west = min(c1[1], c2[1])
north = min(c1[0], c2[0])
east = max(c1[1], c2[1])
south = max(c1[0], c2[0])
bbox = [west, north, east, south]
print(f"bbox:{bbox}" , flush=True)

bbox:[16587035.93650255, -4204394.83644907, 16588691.479969628, -4202027.838289784]


In [41]:
!ls

Code		   Notebooks  README.md  main_chris.sh	paddocks_cleaned.csv
DAESIM_preprocess  Planet_dl  main.sh	 main_test.sh


In [42]:
# Remove any existing generated files
output_file = os.path.join("terrain_tiles.tif")
if os.path.exists(output_file):
    os.remove(output_file)

input_file = os.path.join('DAESIM_preprocess/mapzen_wms.xml')

In [44]:
def run_gdalwarp(bbox, input_file, output_file):
    """Use gdalwarp to download a tif using terrain tiles"""
    min_x, min_y, max_x, max_y = bbox
    
    command = [
        "gdalwarp",
        "-of", "GTiff",
        "-te", str(min_x), str(min_y), str(max_x), str(max_y),
        input_file, output_file
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    
    print("STDOUT:", result.stdout, flush=True)
    print("STDERR:", result.stderr, flush=True)

In [45]:
# Harvest the data
print("Running gdalwarp", flush=True)
run_gdalwarp(bbox, input_file, output_file)

Running gdalwarp
STDOUT: Creating output file that is 347P x 495L.
Processing DAESIM_preprocess/mapzen_wms.xml [1/1] : 0Using internal nodata values (e.g. -32768) for image DAESIM_preprocess/mapzen_wms.xml.
Copying nodata values from source DAESIM_preprocess/mapzen_wms.xml to destination terrain_tiles.tif.
...10...20...30...40...50...60...70...80...90...100 - done.

STDERR: 


In [27]:



# Load the tiff into a numpy array rasterio
with rasterio.open(output_file) as dataset:
    dem = dataset.read(1) 
    meta = dataset.meta.copy()

downsampled_dem = dem.copy()

# There are some clearly bad measurements in terrain tiles and this attempts to assign them np.nan.
threshold = 10
heights = sorted(set(downsampled_dem.flatten()))
lowest_correct_height = min(heights)
for i in range(len(heights)//2, -1, -1):
    if heights[i + 1] - heights[i] > threshold:
        lowest_correct_height = heights[i + 1] 
        break
dem_cleaned = np.where(downsampled_dem < lowest_correct_height, np.nan, downsampled_dem)
Z = dem_cleaned

# Extract into lists for interpolating
x_coords, y_coords = np.meshgrid(np.arange(Z.shape[1]), np.arange(Z.shape[0]))
x_flat = x_coords.flatten()
y_flat = y_coords.flatten()
z_flat = Z.flatten()

# Remove NaN values before interpolating
mask = ~np.isnan(z_flat)
x_flat = x_flat[mask]
y_flat = y_flat[mask]
z_flat = z_flat[mask]
xy_coords = np.vstack((x_flat, y_flat), dtype=float).T

# Replace bad/nan/missing values with the nearest neighbour
X, Y = np.meshgrid(np.linspace(0, Z.shape[1] - 1, Z.shape[1]),
            np.linspace(0, Z.shape[0] - 1, Z.shape[0]))
print(f"Interpolating grid of size {Z.shape}", flush=True)
nearest = griddata(xy_coords, z_flat, (X, Y), method='nearest')
print(f"Size of returned grid {nearest.shape}", flush=True)

# From my tests I couldn't see any improvement from downscaling to 30mx30m
# zoomed = zoom(nearest, 1/3, order=0) 
zoomed = nearest

# Modify the meta as needed, for example if you're saving a single band:
meta.update({
    "driver": "GTiff",
    "height": zoomed.shape[0],
    "width": zoomed.shape[1],
    "count": 1,  # Number of bands
    "dtype": dem.dtype
})

# Save the numpy array back to a new TIFF file
output_tiff = os.path.join("Data/dem.tif")
with rasterio.open(output_tiff, 'w', **meta) as dst:
    dst.write(dem, 1)
print(f"Saved the numpy array of shape {zoomed.shape} to {output_tiff}")

return zoomed


def parse_coord(coord):
  """
  Example inputs:
    Single coord : '-34.737893, 150.530024'
    Corner coords: '-34.737893, 150.530024; -34.737893, 150.530024'
  Example returns:
    Single coord : ['-34.737893', '150.530024']
    Corner coords: [[-34.737893, 150.530024],[-34.737893, 150.530024]]
  """
  print(f"Parsing coord: {coord}")
  if ';' in coord:
    print("Parsing corner coords")
    try:
      if ';' in coord:
        print("found ; in coord")
        coords = coord.split(';')
      print(f"Coord after split: {coords}")
      coords = [c.split(',') for c in coords]
      coords = [[float(c) for c in cs] for cs in coords]
    except:
      coords = None
  else:
    print("Parsing single coord")
    try:
      coords = coord.split(',')
      float(coords[0]),float(coords[1])
    except:
      coords = None

  print(f"Parsed coords: {coords}")
  return coords

def create_bounding_box(lat, lon, distance_km=3):
    """Create a bounding box using the 'epsg:3857' (flat) projection, and convert back to 'epsg:4326' (sphere) projection"""
    print("Started creating the bounding box", flush=True)
    wgs84 = Proj(init='epsg:4326')  # WGS 84
    web_mercator = Proj(init='epsg:3857')  # Web Mercator
    
    # Convert the center point to EPSG:3857
    x_center, y_center = transform(wgs84, web_mercator, lon, lat)
    
    # Calculate the offset in meters (10 km = 10,000 meters)
    offset = distance_km * 1000 / 2  # half distance for each direction
    
    # Define the bounding box and convert back to wgs84
    min_x, min_y = transform(web_mercator, wgs84, x_center - offset, y_center - offset)
    max_x, max_y = transform(web_mercator, wgs84, x_center + offset, y_center + offset)

    coords = [[min_y, min_x], [max_y, max_x]]
    return coords


print("coords: ", coords)
dem = generate_terrain_tiles(coords[0], coords[1])
plt.imshow(dem)
plt.show()

Parsing coord: -35.280716, 149.021031
Parsing single coord
Parsed coords: ['-35.280716', ' 149.021031']
Started creating the bounding box
coords:  [[-35.29171510601581, 149.0075562707382], [-35.26971539981895, 149.03450572926178]]
bbox:[16587445.288408635, -4203595.066255476, 16590445.288408635, -4200595.066255477]
Running gdalwarp
STDOUT: 
STDERR: ERROR 4: mapzen_wms.xml: No such file or directory
ERROR 4: Failed to open source file mapzen_wms.xml




/g/data/xe2/John/geospatenv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/g/data/xe2/John/geospatenv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/jobfs/123114450.gadi-pbs/ipykernel_3430486/4292346436.py:177: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-

RasterioIOError: Data/terrain_tiles.tif: No such file or directory

Parsing coord: -35.280716, 149.021031
Parsing single coord
Parsed coords: ['-35.280716', ' 149.021031']


/g/data/xe2/John/geospatenv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/g/data/xe2/John/geospatenv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/jobfs/123114450.gadi-pbs/ipykernel_3430486/434968372.py:19: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-py

(16587445.288408635,
 -4203595.066255477,
 16590445.288408635,
 -4200595.066255477)

Downloaded output.tif
STDOUT: 
STDERR: ERROR 4: GTiff: No such file or directory
ERROR 4: Failed to open source file GTiff




In [24]:
result

In [7]:
!ls

Code		   Notebooks  README.md  main_chris.sh	paddocks_cleaned.csv
DAESIM_preprocess  Planet_dl  main.sh	 main_test.sh


In [ ]:
def terrain_tiles(lat, lon, buffer, outdir, stub):
    """Download 10m resolution elevation from terrain_tiles"""

In [ ]:





def transform_coords(c1, c2, inputEPSG, outputEPSG):
    x1,y1 = transform_proj(c1[0], c1[1], inputEPSG, outputEPSG)
    x2,y2 = transform_proj(c2[0], c2[1], inputEPSG, outputEPSG)
    return (x1, y1, x2, y2)

def create_bounding_box(lat, lon, distance_km):
    """Use google maps coordinates to create bounding box for Terrain Tiles"""
    wgs84 = Proj(init='epsg:4326')  # WGS 84
    web_mercator = Proj(init='epsg:3857')  # Web Mercator
    
    # Convert the center point to EPSG:3857
    x_center, y_center = transform(wgs84, web_mercator, lon, lat)
    
    # Calculate the offset in meters (10 km = 10,000 meters)
    offset = distance_km * 1000 / 2  # half distance for each direction
    
    # Define the bounding box in EPSG:3857
    min_x = x_center - offset
    max_x = x_center + offset
    min_y = y_center - offset
    max_y = y_center + offset
    
    return min_x, min_y, max_x, max_y


def run_gdalwarp(bbox, input_file, output_file):
    """Use gdalwarp to download a tif using terrain tiles"""
    min_x, min_y, max_x, max_y = bbox
    
    command = [
        "gdalwarp",
        "-of", "GTiff",
        "-te", str(min_x), str(min_y), str(max_x), str(max_y),
        input_file, output_file
    ]
    subprocess.run(command, capture_output=True, text=True)
    
    print("STDOUT:", result.stdout, flush=True)
    print("STDERR:", result.stderr, flush=True)


def generate_terrain_tiles(c1, c2):
    """Harvest 
    params:
        c1 - coordinate 1. e.g. [-35.297579, 149.003879] (coordinate reference system is EPSG:4326, the one use by google maps)
        c2 - coordinate 2. e.g. [-35.280223, 149.018751]
    return:
        dem - a 2d array of floats showing the height at each cell
    """
    # Terrain tiles API requires EPSG:3857
    bbox_3857 = transform_coords(c1, c2, 4326, 3857)
    c1, c2 = (bbox_3857[1], bbox_3857[0]), (bbox_3857[3], bbox_3857[2]) 
    west = min(c1[1], c2[1])
    north = min(c1[0], c2[0])
    east = max(c1[1], c2[1])
    south = max(c1[0], c2[0])
    bbox = [west, north, east, south]
    print(f"bbox:{bbox}" , flush=True)

    # Remove any existing generated files
    output_file = os.path.join("Data/terrain_tiles.tif")
    if os.path.exists(output_file):
        os.remove(output_file)

    input_file = os.path.join('Backend/queries/mapzen_wms.xml')

    # Harvest the data
    print("Running gdalwarp", flush=True)
    run_gdalwarp(bbox, input_file, output_file)

    # Load the tiff into a numpy array rasterio
    with rasterio.open(output_file) as dataset:
        dem = dataset.read(1) 
        meta = dataset.meta.copy()

    downsampled_dem = dem.copy()

    # There are some clearly bad measurements in terrain tiles and this attempts to assign them np.nan.
    threshold = 10
    heights = sorted(set(downsampled_dem.flatten()))
    lowest_correct_height = min(heights)
    for i in range(len(heights)//2, -1, -1):
        if heights[i + 1] - heights[i] > threshold:
            lowest_correct_height = heights[i + 1] 
            break
    dem_cleaned = np.where(downsampled_dem < lowest_correct_height, np.nan, downsampled_dem)
    Z = dem_cleaned

    # Extract into lists for interpolating
    x_coords, y_coords = np.meshgrid(np.arange(Z.shape[1]), np.arange(Z.shape[0]))
    x_flat = x_coords.flatten()
    y_flat = y_coords.flatten()
    z_flat = Z.flatten()

    # Remove NaN values before interpolating
    mask = ~np.isnan(z_flat)
    x_flat = x_flat[mask]
    y_flat = y_flat[mask]
    z_flat = z_flat[mask]
    xy_coords = np.vstack((x_flat, y_flat), dtype=float).T

    # Replace bad/nan/missing values with the nearest neighbour
    X, Y = np.meshgrid(np.linspace(0, Z.shape[1] - 1, Z.shape[1]),
                np.linspace(0, Z.shape[0] - 1, Z.shape[0]))
    print(f"Interpolating grid of size {Z.shape}", flush=True)
    nearest = griddata(xy_coords, z_flat, (X, Y), method='nearest')
    print(f"Size of returned grid {nearest.shape}", flush=True)

    # From my tests I couldn't see any improvement from downscaling to 30mx30m
    # zoomed = zoom(nearest, 1/3, order=0) 
    zoomed = nearest

    # Modify the meta as needed, for example if you're saving a single band:
    meta.update({
        "driver": "GTiff",
        "height": zoomed.shape[0],
        "width": zoomed.shape[1],
        "count": 1,  # Number of bands
        "dtype": dem.dtype
    })

    # Save the numpy array back to a new TIFF file
    output_tiff = os.path.join("Data/dem.tif")
    with rasterio.open(output_tiff, 'w', **meta) as dst:
        dst.write(dem, 1)
    print(f"Saved the numpy array of shape {zoomed.shape} to {output_tiff}")

    return zoomed

